In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import h5py
import numpy as np

In [3]:
f = h5py.File('kitchen.hdf5','r')
data = np.array(f['data_mat'])
#switch the signed distance field data into unsigned distance field data 
data[data<0] = -data[data<0]
#present the shape of the experimental data
print(data[data<0].shape)
print(data[data>=0].shape)
print(data.shape)

(0,)
(260341760,)
(7945, 32, 32, 32)


In [ ]:
#load the trained model first

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
class SAE(nn.Module):
    def __init__(self):
        super(SAE,self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1,out_channels=8,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv2 = nn.Conv3d(in_channels=8,out_channels=32,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv3 = nn.Conv3d(in_channels=32,out_channels=64,kernel_size=2,stride=2,padding=0,bias=False)
        self.conv4 = nn.ConvTranspose3d(in_channels=64,out_channels=32,kernel_size=2,stride=2,padding=0,bias=True)
        self.conv5 = nn.ConvTranspose3d(in_channels=32,out_channels=8,kernel_size=2,stride=2,padding=0,bias=True)
        self.conv6 = nn.ConvTranspose3d(in_channels=8,out_channels=1,kernel_size=2,stride=2,padding=0,bias=True)
    def forward(self,x):
        x1 = F.relu(self.conv1(x))
        x1 = F.relu(self.conv2(x1))
        x1 = F.relu(self.conv3(x1))
        x2 = F.relu(self.conv4(x1))
        x2 = F.relu(self.conv5(x2))
        x2 = F.relu(self.conv6(x2))
        return x1,x2

In [6]:
torch.cuda.empty_cache()

In [7]:
net = SAE().to(device)
net.load_state_dict(torch.load('UDF-AE.pkl'), strict = False)

<All keys matched successfully>

In [16]:
#set the serial number of the baseline streamline
k = 50
o = np.array(data[k])
io = torch.Tensor(o).to(device).view(1,1,32,32,32)
mo1,mo2 = net(io)
co = np.array(mo1.data.cpu()).flatten()
co1 = np.array(mo2.data.cpu()).flatten()

In [17]:
dic = {}

In [18]:
#calculate the Euclidean distance between the remaining streamlines and the baseline streamline in the encoded latent space
for i in range(len(data)):
    m_input = torch.Tensor(np.array(data[i])).to(device).view(1,1,32,32,32)
    i_m,i_out = net(m_input)
    i_c = np.array(i_m.data.cpu()).flatten()
    ans = np.sum((co-i_c)**2)/16**2
    dic[i] = ans

In [19]:
lis = sorted(dic.items(),key = lambda x:x[1])

In [20]:
#print the serial numbers of the top 10 streamlines that are similar to the baseline streamline
print(lis[0:10])

[(50, 0.0), (51, 1670.8907470703125), (59, 3838.50390625), (48, 5201.2177734375), (39, 7379.06982421875), (58, 7635.2734375), (38, 9409.0126953125), (57, 20851.50390625), (320, 25147.8671875), (47, 27416.0)]


In [21]:
out_list = []
for i in range(10):
    out_list.append(lis[i][0])
print(out_list)

[50, 51, 59, 48, 39, 58, 38, 57, 320, 47]


In [13]:
#stored the serial numbers
out_data = np.zeros(7945)
for i in range(len(out_data)):
    if i in out_list:
        out_data[i] = 1
np.save('kitchen_query_10.npy',out_data)

In [16]:
#stored the serial number of the baseline streamline
out_data_1 = np.zeros(7945)
out_data[50] = 1
np.save('kitchen_query_1.npy',out_data_1)